# Recommenders 1

Here we explore simple matrix factorization models on the [smallest movie-lens dataset](https://grouplens.org/datasets/movielens/)

## Quick Recap

On the internet, people rate items (or simply click on them, showing preference).

The matrix of all ratings -- of size (n_user,n_item) -- is highly sparse. Indeed, no one rates everything, people only rate a subset of items.

Recommendation (more specifically **collaborative filtering**) goal is to fill this matrix, by predicting preference or rating.

The most used algorithms for such tasks are matrix factorization ones: it takes advantage from the fact that a matrix can be decomposed into two sub-matrices.

In this practical session, we will explore how to use matrix factorization for collaborative filtering


## Load data

In [ ]:
import numpy as np

# Rating matrices are highly sparse (there are many missing ratings).
# Therefore, the use of sparse matrices is recommended.

from scipy.sparse import dok_matrix

# Data file is a CSV
with open("dataset/ml-latest-small/ratings.csv") as ratings:
    rating_data = ratings.readlines()

# We separate the header from actual data
header = rating_data[0]
print(header)

# Data is CSV: userID,itemID,rating,timestamp
rating_data = rating_data[1:] #contains list(tuple(uid,iid,rating,timestamp))


# Helper to transform one "str" line to a tuple
def line2tuple(l):
    uid,iid,rating, timestamp = l.strip().split(",")
    return (int(uid),int(iid),float(rating), int(timestamp))

# generator for one line. (enables direct unpacking)
def tl(l):
    for x in l:
        yield(line2tuple(x))

# We first remap users and item to ids between (0,len(user)) and (0,len(item))
u_dic = {}
i_dic = {}
        
for uid,iid,rating,ts in tl(rating_data):  # iterating on all data
    uk = u_dic.setdefault(uid,len(u_dic))
    ik = i_dic.setdefault(iid,len(i_dic))

num_user = len(u_dic)
num_item = len(i_dic)

print("there are "+str((num_user,num_item)) +" users and items")


## Factorization on full rating matrix

Factorizing a matrices into two submatrices yields latent representations of users and items. 

In this first part, we aim at visualizing obtained item embeddings.
We propose to use the following function `save_embeddings` and [the tensorflow projector](http://projector.tensorflow.org/) to visualize obtained latent features.

In [ ]:
# This function saves embeddings (a numpy array) and associated labels into tsv files.

def save_embeddings(embs,dict_label,path):
    """
    embs is Numpy.array(N,size)
    dict_label is {str(word)->int(idx)} or {int(idx)->str(word)}
    """
    def int_first(k,v):
        if type(k) == int:
            return (k,v)
        else:
            return (v,k)

    np.savetxt(f"{path}_vectors.tsv", embs, delimiter="\t")

    #labels 
    if dict_label:
        sorted_labs = np.array([lab for idx,lab in sorted([int_first(k,v) for k,v in dict_label.items()])])
        print(sorted_labs)
        with open(f"{path}_metadata.tsv","w") as metadata_file:
            for x in sorted_labs: #hack for space
                if len(x.strip()) == 0:
                    x = f"space-{len(x)}"
                    
                metadata_file.write(f"{x}\n")
                

###  First task: decompose rating matrix with NMF and visualize items

Three things to do:
    - Create sparse matrix of ratings
    - Factorize matrix with NMF into U (user sub-matrix) and I (item sub-matrix)
    - Extract item labels (movie titles) and save embeddings/labels with preceding function 

In [ ]:
from sklearn.decomposition import NMF

# (1) Create sparse matrix from all ratings
Full = dok_matrix((num_user, num_item), dtype=np.float32)

for uid,iid,rating,ts in tl(rating_data):
    Full[u_dic[uid],i_dic[iid]] = float(rating) #don't forget to remap users and items
    

    
# (2) Factorizing matrix

model = NMF(n_components=25, init='random', random_state=0, max_iter=350)
U = model.fit_transform(Full) #users
I = model.components_      #items

I = I.transpose()
I.shape

# (3) Loading labels and saving embeddings + vectors


# data is csv with header
with open("dataset/ml-latest-small/movies.csv") as movies:
    movie_data = movies.readlines()

# print header
print(movie_data[0])

# id-> title dictionnary
movie_names = {}

for x in movie_data[1:]:
    iid, title, *rest =  x.split(',')
    
    iid = float(iid)
    

    if iid in i_dic:
        movie_names[i_dic[iid]] = title


####################################
 
# Saving into "item_50_vectors.tsv" and "item_50_metadata.tsv".
save_embeddings(I,movie_names,"item_50")



[Saved vectors/label can be visualized in tensorflow projector](http://projector.tensorflow.org/)

## Collaborative Filtering  by predicting ratings:

The Netflix competition introduced this framework: the goal is to predict missing ratings using existing ones.


** Building Train/Test set **

In [ ]:
# We take 10% of the train set as test data
train_mat = dok_matrix((num_user, num_item), dtype=np.float32)
test = []
train = []
    
for i,(uid,iid,rating,ts) in enumerate(tl(rating_data)):
    if i%10 == 0: #one out of 10 is for test
        test.append((uid,iid,rating))
    else:
        train.append((uid,iid,rating))
        train_mat[u_dic[int(uid)],i_dic[int(iid)]] = rating
    
print("Number of train examples: ", train_mat.nnz)
print("Number of test examples: ", len(test))


** Evaluating error **:

The goal is to minimize the mean difference between the $n$ predicted and true ratings (respectively $\hat{r_{ui}}, r_{ui}$). 

Two common metrics are the Mean Squared Error (MSE) $$\frac{1}{n}\sum^1_n (r_{ui}-\hat{r_{ui}})^2$$ and the Mean Absolute Error (MAE) $$\frac{1}{n}\sum^1_n abs(r_{ui}-\hat{r_{ui}})$$.

## 1 ) Complete error functions (TODO)

In [ ]:
# take as input a list of differences between prediction and truth.

def MSE_err(truth,pred):
    """
    computes MSE from real-pred difference
    """
    return ########

def MAE_err(truth,pred):
    """
    computes MAE from real-pred difference
    """
    return ###########
        


##  Vanilla NMF AND SVD for rating prediction

First, we test two simple models:
- Non Negative Matrix Factorization
- SVD

The goal is to minimize the following

## $$\min\limits_{U,I}\sum\limits_{(u,i)} (r_{ui} -  I_i^TU_u)^2 $$

in order to have the following **prediction rule**: 

## $$r_{ui} = U_u.I_i $$

## (TODO) What to do: 

- (1) complete prediction function
- (2) fit nmf model and get User and Item profiles 
- (3) fit svd model and get User and Item profiles

In [ ]:
from sklearn.decomposition import NMF, TruncatedSVD


def pred_func(U,I,uid,iid):
    u_p = U[uid,:]
    i_p = I[:,iid]
    
    return ######                                      # TO COMPLETE (see preceding prediction rule)

print("----------------------NMF---------------------------")



## NMF model
model = NMF(n_components=100, solver='cd' ,random_state=0, max_iter=100,alpha=5,l1_ratio=0.5)
#fit and get
U_nmf =  #users profiles                      # TO COMPLETE                                                                      
I_nmf =  #items profiles                       # TO COMPLETE


## Getting the truth values
truth_tr = np.array([rating for (uid,iid),rating in train_mat.items()])
truth_te = np.array([ rating for uid,iid,rating in test])

## Predicting the ratings
prediction_tr = np.array([pred_func(U,I, uid,iid) for (uid,iid),rating in train_mat.items()])
prediction_te = np.array([pred_func(U,I,u_dic[uid],i_dic[iid]) for uid,iid,rating in test])


print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))
    
print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))


print("----------------------SVD---------------------------")

## SVD Model

model = TruncatedSVD(n_components=150)
U_svd =  #users profiles                                             # TO COMPLETE
I_svd =  #items profiles                                             # TO COMPLETE 

## We already have the truth values, we just predict
prediction_tr = np.array([pred_func(U,I, uid,iid) for (uid,iid),rating in train_mat.items()])
prediction_te = np.array([pred_func(U,I,u_dic[uid],i_dic[iid]) for uid,iid,rating in test])


print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))
    
print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))




## Normalization and Regularization


Trying to model the ratings this way is highly prone to overfitting: The train error is low, but the test error is much higher. Traditionnally, models are highly regularized to achieve better performances

$$
\min\limits_{U,I}\sum\limits_{(u,i)} \underbrace{(r_{ui} -  I_i^TU_u)^2}_\text{minimization} + \underbrace{\lambda(||U_u||^2+||I_u||^2)}_\text{regularization}
$$

**Mean normalization** is also a common way to improve performance. In fact, the global mean is the first baseline to try when doing rating prediction


## (TODO) Mean baseline:

To get the mean baseline:

- (1) compute training set mean
- (2) complete prediction function

In [ ]:
print("----------------------MEAN ONLY---------------------------")


# compute mean of list(train_mat.values())
mean = # TO COMPLETE


def pred_func(U,I,uid,iid):    
    return # TO COMPLETE

print("mean rating is ", mean)


## We already have the truth values, we just predict
prediction_tr = np.array([pred_func(U,I, uid,iid) for (uid,iid),rating in train_mat.items()])
prediction_te = np.array([pred_func(U,I,u_dic[uid],i_dic[iid]) for uid,iid,rating in test])


print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))
    
print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))

Indeed, using only the mean ratings to predict future ratings has better performances.

### (TODO) Taking the mean rating into account:

Most work on rating prediction model mean ratings. The goal is to only factor deviation from the mean.

To take into account the mean, the easiest way is to substract it by doing the following:

- (1) compute training set mean
- (2) remove mean from training rating matrix
- (3) factorize the normalized matrix
- (4) when predicting a rating, simply add the mean

The prediction rule becomes:
## $$r_{ui} = \mu + U_u.I_i $$

In [ ]:
from sklearn.decomposition import NMF, TruncatedSVD


# (1) compute mean of list(train_mat.values())
mean = # TO COMPLETE

def pred_func(U,I,uid,iid):
    # TO COMPLETE FULLY
    
    return # TO COMPLETE


# (2) remove mean from training matrix
tmn = dok_matrix((num_user, num_item), dtype=np.float32)

for (uid,iid), rating in train_mat.items():
    tmn[uid,iid] = #TO COMPLETE

# (3) factorize matrix
model_norm = TruncatedSVD(n_components=30)
#fit 
U_msvd =  #users profiles
I_msvd =  #items profiles



## We already have the truth values, we just predict
prediction_tr = np.array([pred_func(U,I, uid,iid) for (uid,iid),rating in train_mat.items()])
prediction_te = np.array([pred_func(U,I,u_dic[uid],i_dic[iid]) for uid,iid,rating in test])


print("Training Error:")
print("MSE:",  MSE_err(prediction_tr,truth_tr))
print("MAE:",  MAE_err(prediction_tr,truth_tr))
    
print("Test Error:")
print("MSE:",  MSE_err(prediction_te,truth_te))
print("MAE:",  MAE_err(prediction_te,truth_te))
    
    


## Collaborative Filtering and Ranking Metrics

Most of the time recommender systems present $k$ items to the users. Therefore, recommenders are often evaluated by how many relevant items are in their $k$ set (using ranking metrics)

Different ranking methods exists such as the [Mean Reciprocal Rank](http://en.wikipedia.org/wiki/Mean_reciprocal_rank) or the [normalized Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)

## (TODO) Let's implement nDCG

In [ ]:


# The dcg@k is the sum of the relevance, penalized gradually
def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        
    """
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        
    return 0.

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# dcg_at_k(r, 1) => 3.0
# dcg_at_k(r, 2) => 4.2618595071429155
    

# And it's normalized version
def ndcg_at_k(r, k):
    """
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    """
    dcg_max = # dcg_at_k(....)  # TO COMPLETE 
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# ndcg_at_k(r, 1) => 1.0
# ndcg_at_k(r, 4) => 0.96519546960144276
    

## (TODO) compute nDCG
What is nDCG for every model we tried ? ?



In [ ]:
from random import shuffle

def group_by_user(tuple_list):
    r_dic = {}
    for uid,iid,rating in tuple_list:
        list_rev = r_dic.get(u_dic[uid],[])
        list_rev.append((u_dic[uid],i_dic[iid],rating))
    
        r_dic[u_dic[uid]] =list_rev
    return r_dic

userg_train = group_by_user(train)
userg_test = group_by_user(test)




def mean_ndcg_UI(U,I):
    mean_ndcg = 0
    num_users = 0
    
    for _,list_rating in userg_train.items():

        pred_ratings = [(pred_func(U,I,uid,iid)) for uid,iid,rating in list_rating]
        real_ratings = [rating for uid,iid,rating in list_rating]
        pred_objects = # TO COMPLETE  tip: inverse argsort: np.argsort(pred_ratings)[::-1]
        

        mean_ndcg += ndcg_at_k(pred_objects,10)
        num_users += 1

    return  mean_ndcg/num_users
    
    
def random_ndcg():
    mean_ndcg = 0
    num_users = 0
    
    for _,list_rating in userg_train.items():

        real_ratings = [rating for uid,iid,rating in list_rating]
        shuffle(real_ratings)
        pred_objects = real_ratings

        mean_ndcg += ndcg_at_k(pred_objects,10)
        num_users += 1

    return  mean_ndcg/num_users
    
print("ndcg nmf", mean_ndcg_UI(U_nmf,I_nmf)) 
print("ndcg svd", mean_ndcg_UI(U_svd,I_svd)) 
print("ndcg svd + mean", mean_ndcg_UI(U_msvd,I_msvd)) 
print("mean == random", random_ndcg())



